In [1]:
import numpy
import time
import ipyparallel as ipp
from scipy.stats import multinomial
from scipy import stats
from joblib import Parallel, delayed,Memory
from qiskit_nature.operators.second_quantization import FermionicOp
from qiskit_nature.problems.second_quantization.electronic.builders import fermionic_op_builder
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
#from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit.opflow import I,X,Y,Z,StateFn,AbelianGrouper
from qiskit.chemistry import FermionicOperator
from qiskit.providers.aer import AerSimulator
from qiskit.aqua.operators.legacy import op_converter
from openfermion.circuits import slater_determinant_preparation_circuit
from qiskit.quantum_info import Pauli
from qiskit import QuantumCircuit,execute,QuantumRegister,ClassicalRegister
from qiskit import Aer
from openfermion.linalg import givens_decomposition_square as givensRotns
from qiskit.opflow.primitive_ops import PauliSumOp
from qiskit.utils import QuantumInstance
from qiskit.opflow import PauliExpectation, CircuitSampler, CircuitStateFn 
import scipy
from joblib import Parallel, delayed
from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.compiler import transpile
from qiskit.circuit.random import random_circuit
from qiskit.chemistry.components.variational_forms import UCCSD

/home/amukh/Anaconda3-Linux/lib/python3.8/site-packages/qiskit/chemistry/__init__.py:170: DeprecationWarning: The package qiskit.chemistry is deprecated. It was moved/refactored to qiskit_nature (pip install qiskit-nature). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/master/README.md#migration-guide>
  warn_package('chemistry', 'qiskit_nature', 'qiskit-nature')


In [6]:
import warnings
warnings. filterwarnings('ignore')

In [7]:
c = ipp.Client()
num_qubits=8

In [8]:
c.ids

[0, 1, 2, 3, 4, 5, 6, 7]

In [9]:
%run Utils.ipynb
%run SMO.ipynb
%run EstimateHamiltonianMean.ipynb

In [10]:
U=7
qubitH,Hmat,Eg,state_g=egBandHamiltonian(U)

In [14]:
qubitH.paulis

[[(-13.659397921999993+0j), Pauli('IIIIIIII')],
 [(2.2640000000000007+0j), Pauli('IIIIIIIZ')],
 [(-0.150812971+0j), Pauli('IIIIIYZY')],
 [(-0.150812971+0j), Pauli('IIIIIXZX')],
 [(2.2640000000000016+0j), Pauli('IIIIIIZI')],
 [(-0.150812971+0j), Pauli('IIIIYZYI')],
 [(-0.150812971+0j), Pauli('IIIIXZXI')],
 [(-0.1616505195000002+0j), Pauli('IIIIIZII')],
 [(-0.1616505195000002+0j), Pauli('IIIIZIII')],
 [(2.2640000000000002+0j), Pauli('IIIZIIII')],
 [(-0.150812971+0j), Pauli('IYZYIIII')],
 [(-0.150812971+0j), Pauli('IXZXIIII')],
 [(2.2640000000000007+0j), Pauli('IIZIIIII')],
 [(-0.150812971+0j), Pauli('YZYIIIII')],
 [(-0.150812971+0j), Pauli('XZXIIIII')],
 [(-0.1616505195000002+0j), Pauli('IZIIIIII')],
 [(-0.1616505195000002+0j), Pauli('ZIIIIIII')],
 [(0.17499999999999993+0j), Pauli('IIIIIIZZ')],
 [(1.75+0j), Pauli('IIIZIIIZ')],
 [(0.7+0j), Pauli('IIZIIIIZ')],
 [(0.525+0j), Pauli('IIYYIIYY')],
 [(0.525+0j), Pauli('IIXXIIYY')],
 [(0.525+0j), Pauli('IIYYIIXX')],
 [(0.525+0j), Pauli('IIXXIIXX

In [ ]:
#Initial state = HF by default, choice='1' generates simple product state 
var_form_base=InitState(U)

In [13]:
#calling qubit UCCSD pool
qubit_pool_UCCSD=qubit_UCCSD_pool(U)
#calling commutator pool 
commutator_pool,commutator_pool_2=commutatorPool(qubitH)

In [15]:
#Hamiltonian grouping for circuit based measurement
eg_h1_0,eg_h1_1,eg_h2_0,eg_h2_1=egBandHamiltonianPartition(U)
qubitH_0,qubitH_1,qubitH_2=qubitHamiltonianGroups(U)
opArr=[qubitH_0,qubitH_1,qubitH_2]
#two body off diags
circ0_1gate,rotH2body=OffDiag2BodyTerms(opArr[2])
#one body off diags
circ0_8gate,rotH1body=OffDiag1BodyTerms(opArr[0])
Ops=[rotH1body,opArr[1],rotH2body]

In [29]:
print(qubitH_1)

-13.659397922 * IIIIIIII
- 0.1616505195000002 * ZIIIIIII
- 0.1616505195000002 * IZIIIIII
+ 2.2640000000000002 * IIZIIIII
+ 2.2640000000000002 * IIIZIIII
+ 0.17499999999999993 * IIZZIIII
- 0.1616505195000002 * IIIIZIII
- 0.1616505195000002 * IIIIIZII
+ 2.2640000000000002 * IIIIIIZI
+ 1.75 * IIZIIIZI
+ 0.7 * IIIZIIZI
+ 2.2640000000000002 * IIIIIIIZ
+ 0.7 * IIZIIIIZ
+ 1.75 * IIIZIIIZ
+ 0.17499999999999993 * IIIIIIZZ


In [28]:
print(Ops[1])

-13.659397922 * IIIIIIII
- 0.1616505195000002 * ZIIIIIII
- 0.1616505195000002 * IZIIIIII
+ 2.2640000000000002 * IIZIIIII
+ 2.2640000000000002 * IIIZIIII
+ 0.17499999999999993 * IIZZIIII
- 0.1616505195000002 * IIIIZIII
- 0.1616505195000002 * IIIIIZII
+ 2.2640000000000002 * IIIIIIZI
+ 1.75 * IIZIIIZI
+ 0.7 * IIIZIIZI
+ 2.2640000000000002 * IIIIIIIZ
+ 0.7 * IIZIIIIZ
+ 1.75 * IIIZIIIZ
+ 0.17499999999999993 * IIIIIIZZ


In [15]:
#Control Hamiltonian at lower U value
eg_h1_0,eg_h1_1,eg_h2_0,eg_h2_1=egBandHamiltonianPartition(U-3)
qubitH_0,qubitH_1,qubitH_2=qubitHamiltonianGroups(U-3)
opArr=[qubitH_0,qubitH_1,qubitH_2]
#two body off diags
circ0_1gate_ctrl,rotH2body=OffDiag2BodyTerms(opArr[2])
#one body off diags
circ0_8gate_ctrl,rotH1body=OffDiag1BodyTerms(opArr[0])
Ops_ctrl=[rotH1body,opArr[1],rotH2body]

In [16]:
def MeasureForGrouping(OpArr):
    Sum1=0
    Sum2=0
    for Op in OpArr:
        coeffs=[Op.primitive.to_list()[i][1] for i in range(len(Op))]
        Sum1=Sum1+numpy.sum(numpy.abs(coeffs))
        Sum2=Sum2+numpy.sum(numpy.sqrt(numpy.sum(numpy.array(numpy.abs(coeffs))*numpy.array(numpy.abs(coeffs)))))
    r=(Sum1/Sum2)**2
    return r

In [20]:
#params for simulator
max_runs=20
tol=1e-4
qr=QuantumRegister(8,'q')

In [18]:
max_runs=7

In [21]:
%timeit Hestimate([])

[-8.04019110e-01 -7.97323845e-01 -8.05272025e-01 -8.02883656e-01
 -8.01317512e-01 -8.00886823e-01 -8.01356666e-01 -8.04293186e-01
 -8.05037104e-01 -8.05193718e-01 -7.99242372e-01 -8.00651901e-01
 -8.01278359e-01 -7.95405319e-01 -8.08051931e-01 -8.04214878e-01
 -8.03627574e-01 -8.04645568e-01 -7.90197891e-01 -8.04293186e-01
 -1.04755395e+01 -1.04701170e+01 -1.04715388e+01 -1.04651528e+01
 -1.04699874e+01 -1.04669767e+01 -1.04709014e+01 -1.04764776e+01
 -1.04801820e+01 -1.04788120e+01 -1.04699371e+01 -1.04773980e+01
 -1.04795343e+01 -1.04623734e+01 -1.04780800e+01 -1.04743315e+01
 -1.04705195e+01 -1.04576955e+01 -1.04659108e+01 -1.04743585e+01
  5.12695313e-04 -2.49023437e-03 -5.85937500e-04 -1.17187500e-03
  0.00000000e+00 -6.59179688e-04  8.78906250e-04 -1.53808594e-03
 -2.92968750e-04  1.31835937e-03 -1.24511719e-03  5.12695313e-04
 -1.46484375e-04  4.02832031e-03 -2.49023437e-03 -8.78906250e-04
  8.78906250e-04  1.46484375e-04  5.12695313e-04  2.27050781e-03]
abs error 0.000496961658

In [20]:
params=[]
EnergyArr=[]
Energy2Arr=[]
numCX=0
CXarr=[]
ExcOps=[]
error=1000
steps=36
stepNo=1
timeArr=[]
t1=time.time()

while error>1e-5 and stepNo<steps:
    #When expanding ansatz we need to provide poolChoice=='1' if qubit UCCSD pool and the default is commutator_pool
    #ordChoice=='1' chooses the next generator that acts on different qubits than the last entry in the sequence,
    #by default the ordering does not matter
    PauliOp=ansatzExpander(ordChoice='1')
    print(PauliOp.paulis[0][1].to_label())
    ExcOps.append(PauliOp.paulis[0][1].to_label())
    with open('../'+str(U)+'/labelsQasm_HF_commutator_poolNew.txt','a') as f:
        print(ExcOps[-1],file=f) 
    params.append(0.0)
    var_form_base.push_hopping_operator(PauliOp)
    #optimize with qasm
    circ=var_form_base.construct_circuit(parameters=params)
    qc=QuantumCircuit(qr,name='ansatz_circuit')
    qc.append(circ.to_instruction(),qr)
    runs=25
    Eestm,MSE,params,Eexact=SMO_optimize_qasm(Hestimate,params,max_runs,tol)
    #optimize with statevector
    #E,params=SMO_optimize(Energy,params,runs,tol)
    numCX=numCX+countCXs(ExcOps[-1])
    CXarr.append(numCX)
    EnergyArr.append(Eestm)
    Energy2Arr.append(Eexact)
    timeArr.append(time.time()-t1)
    error=Energy2Arr[-1]-Eg
    print("convergence energy",EnergyArr[-1],Energy2Arr[-1])
    print("excitationOp",ExcOps[-1])
    print("time elapsed",timeArr[-1])
    stepNo=stepNo+1
    

[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  27 tasks      | elapsed:    3.4s
[Parallel(n_jobs=7)]: Done  56 out of  56 | elapsed:    3.6s finished


IIIZIXZY
5
5
5
**********
 -18.788224967754427 (-18.788335371650774+0j)
looped 1 times
5
5
5
**********
 -18.78842671796598 (-18.788335371650774+0j)
5
5
5
**********
 -18.544647321580488 (-18.54464356163391+0j)
5
5
5
**********
 -17.689818267485265 (-17.689859505024103-1.7763568394002505e-15j)
5
5
5
**********
 -18.912858304985228 (-18.91287547860905+0j)
-18.912858304985228 nan -18.91287547860905 15.131638699996984
inner loop error 0.12443158701924872
looped 2 times
5
5
5
**********
 -18.912818094800922 (-18.91287547860905+0j)
5
5
5
**********
 -18.11727735867525 (-18.117291686397405+0j)
5
5
5
**********
 -18.11720082915598 (-18.117211380260606+0j)
5
5
5
**********
 -18.912770814529416 (-18.912875479620027-8.881784197001252e-16j)
-18.912770814529416 nan -18.912875479620027 29.94853379999404
inner loop error 4.728027150591174e-05
convergence energy -18.912770814529416 -18.912875479620027
excitationOp IIIZIXZY
time elapsed 38.05098140001064


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.


IIZIXZYI


[Parallel(n_jobs=7)]: Done  56 out of  56 | elapsed:    2.7s finished


5
5
5
**********
 -18.91289558278463 (-18.912875479620027-8.881784197001252e-16j)
looped 1 times
5
5
5
**********
 -18.912873336447532 (-18.912875479620027-8.881784197001252e-16j)
5
5
5
**********
 -18.11727432764341 (-18.117253419476086+0j)
5
5
5
**********
 -18.11731893519763 (-18.117249647181925+0j)
5
5
5
**********
 -18.912792382022655 (-18.912875479254488+0j)
-18.912792382022655 7.761087046022396e-05 -18.912875479254488 17.243547700025374
5
5
5
**********
 -18.58383481166597 (-18.583885785658257+1.7763568394002505e-15j)
5
5
5
**********
 -17.79481118926704 (-17.794719957999185-8.881784197001252e-16j)
5
5
5
**********
 -19.013477936702564 (-19.01347093353337+1.7763568394002505e-15j)


KeyboardInterrupt: 

In [55]:
Hestimate(params)

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   3 out of   3 | elapsed:    3.0s finished
[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  30 out of  30 | elapsed:    4.0s finished


(0.0015725109431992124+0j)


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  30 out of  30 | elapsed:    1.7s finished


(0.0034874492080341213+0j)


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  30 out of  30 | elapsed:    1.8s finished


(-0.0018116297738690543+0j)
**********



(-19.16824504372954,
 7.565954988631393e-05,
 (-0.8283264008579091, -18.258203368892364, -0.08171527397927089),
 (7.348356985575867e-11, 7.565936010116465e-05, 1.163015794237856e-10),
 (-19.16818162915017+1.7763568394002505e-15j))

In [31]:
-19.070207449568265+19.07481638558412

0.004608936015856102

In [36]:
Eestm,MSE,params,Eexact=SMO_optimize_qasm(HamiltonianEstm,params,max_runs,tol)

KeyboardInterrupt: 

In [ ]:

t55r 